In [1]:
import pandas as pd

# !pip install pandasql
import pandasql as ps
import math

# Analysis and data transformation for all the files 

In [2]:
# url = [
#     "https://raw.githubusercontent.com/BigTimeStats/beach-volleyball/master/data/full_archive/full_archive.csv"
# ]

big_stat_df_with_nan = pd.read_csv(
    "./data/BigTimeStats.csv",
    index_col=None,
    header=0,
    encoding="latin-1",
    usecols=range(33),
)
big_stat_df_with_nan.shape


(85509, 33)

In [3]:
big_stat_df = big_stat_df_with_nan.dropna(axis=0, thresh=5)

In [4]:
name = "Grigorii Voleanin"
dob = "1967-04-01"
hgt = 77.0


big_stat_df.loc[big_stat_df["l_player1"] == name, "l_p1_hgt"] = hgt
big_stat_df.loc[big_stat_df["l_player2"] == name, "l_p2_hgt"] = hgt

big_stat_df.loc[big_stat_df["l_player1"] == name, "l_p1_birthdate"] = dob
big_stat_df.loc[big_stat_df["l_player2"] == name, "l_p2_birthdate"] = dob


In [5]:
big_stat_df[big_stat_df["l_player2"] == name][
    ["l_player2", "l_p2_hgt", "l_p2_birthdate"]
]


,l_player2,l_p2_hgt,l_p2_birthdate
38825,Grigorii Voleanin,77.0,1967-04-01
40438,Grigorii Voleanin,77.0,1967-04-01
44181,Grigorii Voleanin,77.0,1967-04-01
49342,Grigorii Voleanin,77.0,1967-04-01
53238,Grigorii Voleanin,77.0,1967-04-01


In [6]:
date_columns = ["w_p1_birthdate", "w_p2_birthdate", "l_p1_birthdate", "l_p2_birthdate"]
for col in date_columns:
    big_stat_df[col] = pd.to_datetime(big_stat_df[col], errors="coerce")
    big_stat_df[col] = big_stat_df[col].apply(
        lambda x: x.replace(year=x.year - 100) if x.year > 2015 else x
    )
    big_stat_df["date"] = pd.to_datetime(big_stat_df["date"], errors="coerce")

In [7]:
def extract_scores_and_rounds(score):
    try:
        if pd.isna(score):
            return math.nan, math.nan, math.nan

        rounds = score.count(",") + 1
        score_parts = score.split(",")
        total_win_score = 0
        total_lost_score = 0

        for score_part in score_parts:
            win_score, lost_score = score_part.strip().split("-")
            total_win_score += int(win_score)
            total_lost_score += int(lost_score)

        return total_win_score, total_lost_score, rounds
    except ValueError:
        return math.nan, math.nan, math.nan


big_stat_df[["win_score", "lost_score", "num_rounds"]] = (
    big_stat_df["score"].apply(extract_scores_and_rounds).apply(pd.Series)
)
big_stat_df[["win_score", "lost_score", "num_rounds", "score"]].describe(include="all")

,win_score,lost_score,num_rounds,score
count,84292.000000,84292.000000,84292.000000,85485
unique,NaN,NaN,NaN,6977
top,NaN,NaN,NaN,Forfeit or other
freq,NaN,NaN,NaN,996
mean,46.383370,37.522256,2.321917,NaN
std,6.220353,10.211491,0.471335,NaN
min,15.000000,0.000000,1.000000,NaN
25%,42.000000,30.000000,2.000000,NaN
50%,42.000000,35.000000,2.000000,NaN
75%,52.000000,47.000000,3.000000,NaN


In [8]:
## Connecting by using player name

# big_stat_df.describe()

w_p1_dict = {
    "gender": "gender",
    "w_player1": "player_name",
    "w_p1_birthdate": "DOB",
    "w_p1_hgt": "height_inches",
    "w_p1_country": "player_country",
    "duration": "duration",
    "w_player2": "partner",
}
# w_p1_dict.keys()
w_p1_df = big_stat_df[w_p1_dict.keys()].rename(columns=w_p1_dict)

w_p2_dict = {
    "gender": "gender",
    "w_player2": "player_name",
    "w_p2_birthdate": "DOB",
    "w_p2_hgt": "height_inches",
    "w_p2_country": "player_country",
    "duration": "duration",
    "w_player1": "partner",
}

w_p2_df = big_stat_df[w_p2_dict.keys()].rename(columns=w_p2_dict)

l_p1_dict = {
    "gender": "gender",
    "l_player1": "player_name",
    "l_p1_birthdate": "DOB",
    "l_p1_hgt": "height_inches",
    "l_p1_country": "player_country",
    "duration": "duration",
    "l_player2": "partner",
}

l_p1_df = big_stat_df[l_p1_dict.keys()].rename(columns=l_p1_dict)

l_p2_dict = {
    "gender": "gender",
    "l_player2": "player_name",
    "l_p2_birthdate": "DOB",
    "l_p2_hgt": "height_inches",
    "l_p2_country": "player_country",
    "duration": "duration",
    "l_player1": "partner",
}
l_p2_df = big_stat_df[l_p2_dict.keys()].rename(columns=l_p2_dict)


In [10]:
big_stat_players_df = pd.concat([w_p1_df, w_p2_df, l_p1_df, l_p2_df])
big_stat_players_df.shape

big_stat_players_df.to_csv("./data/out/big_stat_players_df.csv", index=False)

In [11]:
# query = """
#     SELECT BVB_ID, FIVB_Name
#     FROM tb_df_unique
#     WHERE tb_df_unique.FIVB_Name not in (
#         SELECT player_name
#         FROM big_stat_players_df )
# """

# # Execute the SQL query
# not_merged_players_df = ps.sqldf(query, globals())

# # Display the players who didn't merge into allplayers_df
# print(not_merged_players_df)


In [12]:
# # Remove duplicates from the DataFrame
# big_stat_players_df.drop_duplicates(subset="player_name", inplace=True)
# big_stat_players_df.reset_index(drop=True, inplace=True)

# print(big_stat_players_df.player_name.describe())

In [13]:
# big_stat_players_df[big_stat_players_df.duplicated(subset="player_name", keep=False)]

In [285]:
## find ids for the players by name and birthdate , assign the ids
## if there is no birthdate, check for player and height
## else just first occurance of the name

# big_stat_players_df vs allplayers_df

# allplayers_df.describe(include="all")  # player_name DOB player_id

# transform allplayers_df.DOB


In [14]:
big_stat_players_df_unique = big_stat_players_df[
    ["gender", "player_name", "DOB", "height_inches"]
].drop_duplicates()
big_stat_players_df_unique.shape


(10089, 4)

In [509]:
# # # Merge big_stat_players_df with allplayers_copy
merged_players_df = pd.merge(
    allplayers_copy,
    big_stat_players_df_unique,
    left_on="big_stat_player_name",
    right_on="player_name",
    how="inner",
)

merged_players_df.rename(columns={"player_id": "BVB_ID"}, inplace=True)
merged_players_df.to_csv("./data/out/merged_players.csv", index=False)

In [510]:
merged_players_df.shape


(10021, 22)

In [15]:
def transform_duration(str):
    try:
        (hours, min) = str.split(":")
        duration = int(hours) * 60 + int(min)
        return duration
    except Exception:
        return math.nan


big_stat_players_df["duration_int"] = big_stat_players_df["duration"].apply(
    transform_duration
)

In [16]:
# big_stat_players_df.to_csv("./data/out/big_stat_players_df.csv", index=False)

In [18]:
average_duration = (
    big_stat_players_df.groupby(["player_name", "DOB"])["duration_int"]
    .mean()
    .reset_index()
)
average_duration.rename(columns={"duration_int": "duration_avg"}, inplace=True)

median_duration = (
    big_stat_players_df.groupby(["player_name", "DOB"])["duration_int"]
    .median()
    .reset_index()
)
median_duration.rename(columns={"duration_int": "duration_median"}, inplace=True)

# Calculate the number of distinct partners for each player (player_name, dob)
distinct_partners_count = (
    big_stat_players_df.groupby(["player_name", "DOB"])["partner"]
    .nunique()
    .reset_index()
)
distinct_partners_count.rename(columns={"partner": "partners_count"}, inplace=True)

# Merge 'average_duration' and 'distinct_partners_count' based on 'player_name' and 'DOB'
duration_partners_df = pd.merge(
    average_duration,
    distinct_partners_count,
    on=["player_name", "DOB"],
)

all_duration_partners_df = pd.merge(
    duration_partners_df,
    median_duration,
    on=["player_name", "DOB"],
)

# all_duration_partners_df.describe()


In [25]:
# Assuming distinct_partners_count is a Django QuerySet object
# Assuming distinct_partners_count is a pandas DataFrame
# Assuming distinct_partners_count is a pandas DataFrame
distinct_partners_count.sort_values(by="partners_count", ascending=False)

,player_name,DOB,partners_count
4345,Justin Phipps,1978-05-09,61
65,Adam Roberts,1976-01-25,44
4096,Jon Mesko,1977-03-30,38
8517,Ty Loomis,1979-05-01,36
7245,Robert deAurora,1981-03-28,35
...,...,...,...
5924,Megan Knepp,1987-12-21,1
466,Anastasia Lysenko,1997-11-28,1
5926,Megan McNamara,1997-08-01,1
3182,Helene Rancourt,1991-04-13,1
